In [1]:
!git clone https://github.com/suryajayaraman/pcdetModules.git

Cloning into 'pcdetModules'...
remote: Enumerating objects: 131, done.
remote: Counting objects: 100% (131/131), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 131 (delta 43), reused 115 (delta 29), pack-reused 0
Receiving objects: 100% (131/131), 1.62 MiB | 5.46 MiB/s, done.
Resolving deltas: 100% (43/43), done.


In [3]:
import torch
from torch.utils.cpp_extension import load

In [4]:
iou3d_nms_cuda = load(name="iou3d_nms_cuda", verbose=False,
                    sources=[
                        '/kaggle/working/pcdetModules/iou3d_nms_cuda/iou3d_cpu.cpp',
                        '/kaggle/working/pcdetModules/iou3d_nms_cuda/iou3d_nms_api.cpp',
                        '/kaggle/working/pcdetModules/iou3d_nms_cuda/iou3d_nms.cpp',
                        '/kaggle/working/pcdetModules/iou3d_nms_cuda/iou3d_nms_kernel.cu',
                    ])


roiaware_pool3d = load(name="roiaware_pool3d", verbose=False,
                    sources=[
                        '/kaggle/working/pcdetModules/roiaware_pool3d/roiaware_pool3d.cpp',
                        '/kaggle/working/pcdetModules/roiaware_pool3d/roiaware_pool3d_kernel.cu',
                    ])

roipoint_pool3d = load(name="roipoint_pool3d", verbose=False,
                    sources=[
                        '/kaggle/working/pcdetModules/roipoint_pool3d/roipoint_pool3d.cpp',
                        '/kaggle/working/pcdetModules/roipoint_pool3d/roipoint_pool3d_kernel.cu',
                    ])


pointnet2_batch = load(name="pointnet2_batch", verbose=False,
                    sources=[
                        '/kaggle/working/pcdetModules/pointnet2_batch/pointnet2_api.cpp',
                        '/kaggle/working/pcdetModules/pointnet2_batch/ball_query.cpp',
                        '/kaggle/working/pcdetModules/pointnet2_batch/ball_query_gpu.cu',
                        '/kaggle/working/pcdetModules/pointnet2_batch/group_points.cpp',
                        '/kaggle/working/pcdetModules/pointnet2_batch/group_points_gpu.cu',
                        '/kaggle/working/pcdetModules/pointnet2_batch/interpolate.cpp',
                        '/kaggle/working/pcdetModules/pointnet2_batch/interpolate_gpu.cu',
                        '/kaggle/working/pcdetModules/pointnet2_batch/sampling.cpp',
                        '/kaggle/working/pcdetModules/pointnet2_batch/sampling_gpu.cu',                    
                    ])

pointnet2_stack = load(name="pointnet2_stack", verbose=False,
                    sources=[
                            '/kaggle/working/pcdetModules/pointnet2_stack/pointnet2_api.cpp',
                            '/kaggle/working/pcdetModules/pointnet2_stack/ball_query.cpp',
                            '/kaggle/working/pcdetModules/pointnet2_stack/ball_query_gpu.cu',
                            '/kaggle/working/pcdetModules/pointnet2_stack/group_points.cpp',
                            '/kaggle/working/pcdetModules/pointnet2_stack/group_points_gpu.cu',
                            '/kaggle/working/pcdetModules/pointnet2_stack/sampling.cpp',
                            '/kaggle/working/pcdetModules/pointnet2_stack/sampling_gpu.cu', 
                            '/kaggle/working/pcdetModules/pointnet2_stack/interpolate.cpp', 
                            '/kaggle/working/pcdetModules/pointnet2_stack/interpolate_gpu.cu',
                            '/kaggle/working/pcdetModules/pointnet2_stack/voxel_query.cpp', 
                            '/kaggle/working/pcdetModules/pointnet2_stack/voxel_query_gpu.cu',
                            '/kaggle/working/pcdetModules/pointnet2_stack/vector_pool.cpp',
                            '/kaggle/working/pcdetModules/pointnet2_stack/vector_pool_gpu.cu'                   
                    ])

In [5]:
import iou3d_nms_cuda
import roiaware_pool3d
import roipoint_pool3d
import pointnet2_batch
import pointnet2_stack

In [ ]:
# !mkdir -p /kaggle/working/outputDir
# !mkdir -p prebuilt_modules/iou3d_nms_cuda
# !ls prebuilt_modules/

# !tar -xvf pcdetModules/iou3d_nms_cuda_p100.tar.gz -C prebuilt_modules/iou3d_nms_cuda/
# !ls prebuilt_modules/iou3d_nms_cuda/tmp/testDir/iou3d_nms_cuda/
# !cp -r /root/.cache/torch_extensions/py37_cu113/iou3d_nms_cuda/ /tmp/testDir/
# !ls /tmp/testDir/iou3d_nms_cuda/
# !tar -zcvf /kaggle/working/iou3d_nms_cuda_p100.tar.gz /tmp/testDir/iou3d_nms_cuda
# !cp -r /root/.cache/torch_extensions/py37_cu113/iou3d_nms_cuda/io

In [ ]:
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") 

boxes_a = torch.Tensor(10,7).to(device)
boxes_b = torch.Tensor(10,7).to(device)

assert boxes_a.shape[1] == boxes_b.shape[1] == 7
ans_iou = torch.cuda.FloatTensor(torch.Size((boxes_a.shape[0], boxes_b.shape[0]))).zero_()

iou3d_nms_cuda.boxes_iou_bev_gpu(boxes_a.contiguous(), boxes_b.contiguous(), ans_iou)